<a href="https://colab.research.google.com/github/Supriyo760/CourseSellApp/blob/master/Supriyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

data set of loader

In [ ]:
!pip install osmnx
import pandas as pd
import requests
import osmnx as ox
def fetch_openaq_data(city, params):
  url = f"https://api.openaq.org/v2/measurements?city={city}"
  response = response.get(url, params= params)
  data = response.json()['results']
  return pd.DataFrame(data)

def fetch_weather_data(lat, lon, api_key):
  url = f"https://api.openweathermap.org/data/2.5/weather"
  params = {'lat': lat, 'lon': lon, 'appid': api_key}
  response = requests.get(url, params=params)
  return response.json()

def get_location_features(lat, lon, dist= 1000):
  G= ox.graph_from_point((lat, lon), dist= dist, network_type= 'drive')
  roads= ox.geometries.geometries_from_point((lat, lon), tags= {'highway': True}, dist=dist)
  factories= ox.geometries.geometries_from_point((lat, lon), tags= {'landuse': 'industrial'}, dist= dist)
  return {'roads': roads, 'factories': factories}


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.5 MB/s eta 0:00:00


data cleaning


In [ ]:
def clean_pollution_data(df):
  df = df.drop_duplicates()
  df = df.dropna(subset=['value', 'coordinates.latitude', 'coordinates.longitude'])
  df['value']= pd.to_numeric(df['value'])
  df['timestamp']= pd.to_datetime(df['date']['utc'])
  #impute missing values
  df= df.fillna(df.mean())
  return df
def features_engineering(df):
  for col in['value']:
    df[col]= (df[col]-df[col].mean())/ df[col].std()
    df['hour']= df['timestamp'].dt.hour
    df['dayofweek']= df['timestamp'].dt.dayofweek
    df['month']= df['timestamp'].dt.month
    return df

SOURCE LABELLING AND SIMULATION


In [ ]:
def label_sources(df):
  df['source']= 'Unknown'
  df.loc[(df['near_main_road']==1)&(df['NO2']>40), 'source'] = 'Vehicular'
  df.loc[(df['near_factory']==1)&(df['SO2']>20), 'source'] = 'Industrial'
  df.loc[(df['near_farmland']==1)&(df['season']=='Dry')& (df['PM2.5']>70), 'source'] = 'Agricultural'
  return df

Model Training and Source Prediction

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

def train_predict_model(df):
  features= ['PM2.5','NO2','SO2','CO','roads_proximity','factories_proximity','temperature','humidity','hour','dayofweek','month']
  x=df[features]
  y=df['source']
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)
  cif= RandomForestClassifier()
  param_grid= {'n_estimators': [50, 100], 'max_depth': [5,10,None]}
  grid= GridSearchCV(cif, param_grid)
  grid.fit(x_train, y_train)
  y_pred= grid.predict(x_test)
  print(classification_report(y_test, y_pred))
  return grid.best_estimator_

Geospatial Mapping and Visualization

In [ ]:
import folium

def plot_heatmap(df):
    m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)
    for _, row in df.iterrows():
        folium.Circle(
            location=[row['latitude'], row['longitude']],
            radius=50,
            color="red" if row['source'] == "Industrial" else "blue",
            fill=True
        ).add_to(m)
    return m


real-time dashboard and alerts(streamlit)

In [ ]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.8 MB/s eta 0:00:00


In [ ]:
import streamlit as st
!pip install streamlit


st.title("AI-Powered Pollution Source Identifier")

city = st.text_input("Enter city")
if st.button("Analyze"):
    st.write(f"Analyzing pollution sources for city: {city}")

2025-09-04 13:56:21.287 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-04 13:56:21.651 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-04 13:56:21.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-04 13:56:21.657 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-04 13:56:21.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-04 13:56:21.659 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-04 13:56:21.659 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-04 13:56:21.660 Thread 'MainThread': mi

In [ ]:
!npm install localtunnel

!streamlit run /path/to/yout/streamlit_app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸/bin/bash: line 1: streamlit: command not found
⠙your url is: https://weak-books-jump.loca.lt
/content/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:21411 (check your firewall settings)
    at Socket.<anonymous> (/content/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙